## 1.1 层和块
![](/img/pytorch/layer-block.png)
一个单独的层接收输入（由前一层提供）， 生成输出（到下一层的输入），并且具有一组可调参数， 这些参数根据从下一层反向传播的信号进行更新

而“比单个层大”但“比整个模型小”的组件就是块（block），可以描述单个层、由多个层组成的组件或整个模型本身。

任何块都必须：
1. 接收输入 → 前向传播 → 产生输出
2. 存储可学习参数
3. 能够计算梯度（PyTorch自动实现）

In [13]:
import torch
from torch import nn
from torch.nn import functional as F

net0 = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20) # 2个样本，每个样本20维
net0(X)

tensor([[ 0.1322, -0.1334,  0.1632, -0.0659,  0.1460,  0.3331,  0.0089,  0.2311,
         -0.1539, -0.0593],
        [ 0.1830, -0.0143,  0.1394,  0.0562,  0.0701,  0.3735, -0.0094,  0.1769,
         -0.1519,  0.0720]], grad_fn=<AddmmBackward0>)

在这个例子中，我们通过实例化nn.Sequential来构建我们的模型， 层的执行顺序是作为参数传递的。 简而言之，nn.Sequential定义了一种特殊的Module， 即在PyTorch中表示一个块的类， 它维护了一个由Module组成的有序列表

### 1.1.1 基础块实现模板
1. 将输入数据作为其前向传播函数的参数。
2. 通过前向传播函数来生成输出。
> 输出的形状可能与输入的形状不同。例如，我们上面模型中的第一个全连接的层接收一个20维的输入，但是返回一个维度为256的输出。
3. 计算其输出关于输入的梯度，可通过其反向传播函数进行访问。通常这是自动发生的。
4. 存储和访问前向传播计算所需的参数。
5. 根据需要初始化模型参数。

In [3]:
class basicBlock(nn.Module): # 继承自`nn.Module`
    def __init__(self):
        super().__init__()   # 让父类初始化
        self.layer1 = nn.Linear(20,256)
        self.layer2 = nn.Linear(256, 10)

    def forward(self, x):    # 定义数据流
        x = self.layer1(x)
        x = F.relu(x)        # relu在nn.functional里定义好了
        x = self.layer2(x)
        return x

net = basicBlock()
net(X)

tensor([[-0.0041, -0.1088,  0.1667, -0.1542,  0.2514, -0.0965,  0.0503,  0.2167,
         -0.2589, -0.1554],
        [-0.0728, -0.0092,  0.1135, -0.1316,  0.2854, -0.0674,  0.0683,  0.0656,
         -0.1738, -0.0804]], grad_fn=<AddmmBackward0>)

### 1.1.2 顺序块
`Sequential`的设计是为了把其他模块串起来，自己从零开始设计只需要定义两个关键函数：
1. 一种将块逐个追加到列表中的函数；
2. 一种前向传播函数，用于将输入按追加块的顺序传递给块组成的“链条”。

In [7]:
class mySequential(nn.Module):
    def __init__(self, *modules):     # \*modules 参数是可变长度的位置参数，接收任意多个nn.Module子类实例
                                      # 允许在创建 MySequential 实例时，传入任意多个参数（比如示例中的 3 个网络层）
                                      # 这些传入的参数会被自动打包成一个元组，在 __init__ 方法内部可以通过 modules 这个变量访问
                                      # MySequential(nn.Linear(20,256), nn.ReLU(), nn.Linear(256,10))
        super().__init__()
        # enumerate(modules)：同时遍历modules的「索引」和「元素」，i是索引（从0开始），module是传入的每个网络层
        for i, module in enumerate(modules):
            # self._modules 是 nn.Module 类内置的有序字典（OrderedDict），专门用来管理子模块
            # PyTorch 会自动识别 _modules 中的子模块，为它们注册参数（比如 nn.Linear 的权重 weight、偏置 bias）
            # 否则这些参数不会被 model.parameters() 识别，也无法用 GPU 训练
            self._modules[str(i)] = module
            # _modules 会保持模块的添加顺序，保证 forward 中按顺序执行
            # _modules 的 key 必须是字符串（不能是整数）

    def forward(self, x):
        for module in self._modules.values():  # 取出 _modules 字典中所有的模块（按添加顺序，毕竟_module的类型是OrderedDict）
            x = module(x)
        return x
net = mySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.1163, -0.0657,  0.3006,  0.2966, -0.0174,  0.1063, -0.0779,  0.1405,
          0.1242,  0.0017],
        [ 0.0988, -0.0893,  0.1842,  0.1976, -0.0580,  0.0897, -0.1011,  0.1267,
         -0.0178,  0.0997]], grad_fn=<AddmmBackward0>)

### 1.1.3 灵活的前向传播
Sequential类使模型构造变得简单， 允许我们组合新的架构，而不必定义自己的类。 然而，并不是所有的架构都是简单的顺序架构。 当需要更强的灵活性时，我们需要定义自己的块。

我们可能希望在前向传播函数中执行Python的控制流，亦或者是执行任意的数学运算， 而不是简单地依赖预定义的神经网络层。

In [8]:
class FlexibleMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(20, 20)
        self.fixed_weight = torch.rand((20, 20), requires_grad=False)
        # 这里的 fixed_weight 是一个固定的权重矩阵，不参与梯度更新

    def forward(self, X):
        # 正常层计算
        X = self.fc(X)
        # 使用固定参数进行计算
        X = F.relu(torch.mm(X, self.fixed_weight) + 1)
        # 条件控制流
        while X.abs().sum() > 1:
            X /= 2
        # 复用层，这相当于两个全连接层共享参数
        X = self.fc(X)
        return X.sum()

net = FlexibleMLP()
net(X)

tensor(-1.1924, grad_fn=<SumBackward0>)

在上面的例子中，隐藏层的权重`self.fixed_weight`在实例化时被随机初始化，之后为常量。这个权重不是一个模型参数，因此它永远不会被反向传播更新。 然后，神经网络将这个固定层的输出通过一个全连接层。

控制流方面，在L1范数大于1时，将输出向量除以2来归一化，迭代直到L1范数小于等于1。

### 1.1.4 嵌套块组合
模块的三个层级:
1. 原子层：nn.Linear, nn.Conv2d 等
2. 组合块：多个层组成的模块（如残差块）
3. 完整模型：多个块组成的网络

In [10]:
class NestedBlock(nn.Module):
    def __init__(self):
        super().__init__()

        # 首先来一个Sequential块
        self.inner_net = nn.Sequential(
          nn.Linear(20, 64),
          nn.ReLU(),
          nn.Linear(64, 32)
        )
        # 然后来一个全连接层
        self.final_layer = nn.Linear(32, 10)

    def forward(self, x):
        # 先通过内部的Sequential块
        x = self.inner_net(x)
        # 然后通过全连接层
        x = self.final_layer(x)
        return x

- 继承nn.Module的规则:
  - 必须调用super().__init__()
  - 在__init__中定义所有层（PyTorch需要知道有哪些参数）
  - forward方法定义实际计算图

- PyTorch会自动追踪forward中的操作以计算梯度
- 避免在__init__之外动态创建层（会让参数管理混乱）
- 使用_modules字典管理子模块（PyTorch的约定）

## 1.2 参数管理
Parameter是Tensor的子类，前者自动require_grad=True，会被nn.Module自动识别和注册，可以通过model.parameters()访问
### 1.2.1 参数访问
首先看看我们最开始的`net0`的第三层的参数（weight和bias）长啥样：

In [ ]:
print(net0[2].state_dict())

OrderedDict({'weight': tensor([[ 0.0323,  0.0240, -0.0257,  ...,  0.0391,  0.0541,  0.0190],
        [-0.0080,  0.0535,  0.0528,  ...,  0.0232,  0.0200,  0.0448],
        [-0.0446,  0.0165,  0.0061,  ...,  0.0485,  0.0176, -0.0484],
        ...,
        [-0.0274,  0.0560,  0.0016,  ...,  0.0461,  0.0403, -0.0311],
        [ 0.0163, -0.0020, -0.0253,  ...,  0.0451, -0.0222, -0.0060],
        [ 0.0579, -0.0501,  0.0479,  ...,  0.0100, -0.0601,  0.0281]]), 'bias': tensor([ 0.0287, -0.0608,  0.0504, -0.0249, -0.0030,  0.0566, -0.0346,  0.0608,
        -0.0102, -0.0239])})


In [ ]:
print(type(net0[2].bias))
print(net0[2].bias)
print(net0[2].bias.data)
net0[2].weight.grad == None # 由于我们还没有调用反向传播，所以参数的梯度处于初始状态


<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([ 0.0287, -0.0608,  0.0504, -0.0249, -0.0030,  0.0566, -0.0346,  0.0608,
        -0.0102, -0.0239], requires_grad=True)
tensor([ 0.0287, -0.0608,  0.0504, -0.0249, -0.0030,  0.0566, -0.0346,  0.0608,
        -0.0102, -0.0239])


True

In [20]:
# 查看参数形状
for name, param in net0[0].named_parameters():
    print(f"{name}: {param.shape}")

# 查看所有参数
for name, param in net0.named_parameters():
    print(f"{name}: {param.shape}")
print(net0)

weight: torch.Size([256, 20])
bias: torch.Size([256])
0.weight: torch.Size([256, 20])
0.bias: torch.Size([256])
2.weight: torch.Size([10, 256])
2.bias: torch.Size([10])
Sequential(
  (0): Linear(in_features=20, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


### 1.2.2 参数初始化
默认情况下，PyTorch会根据一个范围均匀地初始化权重和偏置矩阵， 这个范围是根据输入和输出维度计算出的。 
PyTorch的`nn.init`模块提供了多种预置初始化方法。

In [ ]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        # 标准正态分布初始化
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

# 或是直接初始化
nn.init.xavier_normal_(net0[0].weight) # Xavier初始化是一种常用的初始化方法，它根据输入和输出的维度来缩放权重，使得前向传播和反向传播的方差相等。
nn.init.constant_(net0[0].bias, 0.5) # 常量初始化

# 还能直接修改参数
net0[0].weight.data[:] += 1
net0[0].weight.data[0, 0] = 2


### 1.2.3 参数绑定
有时我们希望在多个层间共享参数，比如说：
- RNN/LSTM：时间步之间共享参数
- 多任务学习：共享底层特征提取器
- Siamese网络：共享编码器
- 注意力机制：共享QKV变换矩阵

In [ ]:
# 创建共享层
shared_layer = nn.Linear(8, 8)

# 在多个位置使用同一层
net = nn.Sequential(
    nn.Linear(4, 8),
    nn.ReLU(),
    shared_layer,  # 第2层
    nn.ReLU(),
    shared_layer,  # 第4层（共享参数）
    nn.ReLU(),
    nn.Linear(8, 1)
)

# 验证参数共享
print(net[2].weight.data[0] == net[4].weight.data[0])  # 全True

# 修改一处，另一处也变化
net[2].weight.data[0, 0] = 100
print(net[4].weight.data[0, 0])  # 也变为100

# 注意：共享参数的梯度会累加！
# 假设我们在第2层和第4层都进行了前向传播和反向传播
# 那么第2层的梯度会累加到第4层的梯度上
# 这在某些情况下是有用的，比如在Siamese网络中共享编码器

tensor([True, True, True, True, True, True, True, True])
tensor(100.)


### 1.2.4 冻结/解冻参数

In [22]:
# 冻结前两层参数
for param in net0[:2].parameters():
    param.requires_grad = False

# 只训练最后一层
optimizer = torch.optim.SGD(
    filter(lambda p: p.requires_grad, net0.parameters()),
    lr=0.01
)

# 解冻参数
for param in net0.parameters():
    param.requires_grad = True

### 1.2.5 参数保存与加载

In [ ]:
# 保存参数
torch.save(net0.state_dict(), 'model_params.pth')

# 加载参数
net0.load_state_dict(torch.load('model_params.pth'))

# 保存整个模型
torch.save(net0, 'full_model.pth')
loaded_net = torch.load('full_model.pth')

## 1.3 自定义层


### 1.4 读写文件

### 1.5 GPU